<a href="https://colab.research.google.com/github/IgorTFromK/GoogleSpeechCommandClassifier/blob/master/train_le_net.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [107]:
import tensorflow as tf
from tensorflow import keras
import numpy as np
import time

FILE_DIR_MAIN = "/content/google_speech_command/"
FILE_DIR_GRIDSEARCH = "/content/google_speech_command/gridsearch/"
FILE_DIR_TRAIN = FILE_DIR_MAIN + "train/"
FILE_DIR_TEST = FILE_DIR_MAIN + "test/"
FILE_DIR_EVAL = FILE_DIR_MAIN + "eval/"

# Hyperparameter for model optimization
NUM_DENSE_LAYER = 1
NUM_NEURONS = [64, 128, 256]
NUM_MEL_BINS = [64, 96]
DROPOUT_RATE = [0.2, 0.3, 0.4]
LEARNING_RATE = [0.01, 0.001]
TRAININGS_EPOCHS = [32, 64, 128]
BATCH_SIZE = [30, 50]

class LeNet():
  def __init__(self, num_dense_layer, num_hidden_neurons, num_mel_bins, dropout_rate,
               learning_rate, input_size):
    self.num_dense_layer = num_dense_layer
    self.num_hidden_neurons = num_hidden_neurons
    self.num_mel_bins = num_mel_bins
    self.dropout_rate = dropout_rate
    self.learning_rate = learning_rate
    self.input_size = input_size


# **Model definition**

In [110]:
# computes log mel spectrogram
def stft_and_log_mel_spectrogram(x, sampling_rate: int, num_mel_spects: int, 
                                  window_size: int, hop_size: int):
  
  stfts = tf.signal.stft(x, frame_length=window_size, frame_step=hop_size, fft_length=window_size)
  spectrograms = tf.abs(stfts)

  # Warp the linear scale spectrograms into the mel-scale.
  num_spectrogram_bins = stfts.shape[-1]
  lower_edge_hertz, upper_edge_hertz, num_mel_bins = 0, sampling_rate / 2, num_mel_spects
  linear_to_mel_weight_matrix = tf.signal.linear_to_mel_weight_matrix(
    num_mel_bins, num_spectrogram_bins, sampling_rate, lower_edge_hertz,
    upper_edge_hertz)
  mel_spectrograms = tf.tensordot(
    spectrograms, linear_to_mel_weight_matrix, 1)
  mel_spectrograms.set_shape(spectrograms.shape[:-1].concatenate(
    linear_to_mel_weight_matrix.shape[-1:]))

  # Compute a stabilized log to get log-magnitude mel-scale spectrograms.
  log_mel_spectrograms = tf.math.log(mel_spectrograms + 1e-6)

  return tf.reshape(log_mel_spectrograms, shape=[-1, log_mel_spectrograms.shape[1], num_mel_spects, 1])

# creates model 
def create_model(leNet: LeNet):
  inputs = keras.Input(shape= leNet.input_size, dtype=tf.float32)
  log_mel_spec = keras.layers.Lambda(stft_and_log_mel_spectrogram, arguments={
      'sampling_rate': 16000, 'num_mel_spects': leNet.num_mel_bins, 
      'window_size': 1024, 'hop_size': 512
  })(inputs)
  conv_1 = keras.layers.Conv2D(filters=8, kernel_size=(3, 3), padding="same")(log_mel_spec)
  batch_1 = keras.layers.BatchNormalization(axis=3)(conv_1)
  activation_1 = keras.layers.Activation('relu')(batch_1)
  pool_1 = keras.layers.MaxPooling2D(pool_size=(3, 2))(activation_1)

  conv_2 = keras.layers.Conv2D(filters=16, kernel_size=(3, 3), padding="same")(pool_1)
  batch_2 = keras.layers.BatchNormalization(axis=3)(conv_2)
  activation_2 = keras.layers.Activation('relu')(batch_2)
  pool_2 = keras.layers.MaxPooling2D(pool_size=(3, 2))(activation_2)

  conv_3 = keras.layers.Conv2D(filters=32, kernel_size=(3, 3), padding="same")(pool_2)
  activation_3 = keras.layers.Activation('relu')(conv_3)
  batch_3 = keras.layers.BatchNormalization(axis=3)(activation_3)

  dropout = keras.layers.Dropout(leNet.dropout_rate)(batch_3)
  flatten = keras.layers.Flatten()(dropout)
  layer_list = []
  layer_list.append(keras.layers.Dense(leNet.num_hidden_neurons[0], activation="relu")(flatten))
  layer_list.append(keras.layers.Dropout(rate=leNet.dropout_rate)(layer_list[-1]))

# Fully connected layer
  for x in range(1, leNet.num_dense_layer - 1):
    layer_list.append(keras.layers.Dense(leNet.num_hidden_neurons[x], activation="relu")(
    layer_list[-1]))  
    layer_list.append(keras.layers.Dropout(rate=leNet.dropout_rate)(layer_list[-1]))
  if leNet.num_dense_layer != 1:
    layer_list.append(keras.layers.Dense(leNet.num_hidden_neurons[-1], activation="relu")(layer_list[-1]))
    layer_list.append(keras.layers.Dropout(rate=leNet.dropout_rate)(layer_list[-1]))

    predictions = keras.layers.Dense(leNet.prediction_classes, activation="softmax")(layer_list[-1])  # Classification layer or output layer

    adam = keras.optimizers.Adam(learning_rate=leNet.learning_rate)
    model = keras.Model(inputs=inputs, outputs=predictions)

    model.compile(optimizer=adam, loss='categorical_crossentropy',metrics=[keras.metrics.TruePositives(), keras.metrics.Precision(), "accuracy"])
    model.summary()

    return model


In [ ]:
def train_model():
  pass

In [ ]:
def test_model():
  pass

In [ ]:
def long_op():
  for x in range(1000):
    time.sleep(2.4)

In [ ]:
def write_file_test(filename:str):
  with open(filename, 'w') as writefile:
    for x in range(10):
      writefile.write("Line: {:d}\n".format(x))

In [103]:
def read_file_test(filename: str):
  with open(filename, 'r') as readfile:
    for line in readfile:
      print(line)